In [1]:
import argparse
import numpy as np
import os.path
import scipy.linalg as sla
import sys
import datetime
import os
import psutil
import sparse

from scipy.sparse import coo_matrix

from dask.distributed import Client
import dask.array as da

from sklearn.datasets import make_sparse_coded_signal

In [2]:
#Inputs
'''
These inputs were copied from the pyspark version.
I may not need all of these.

'''


input_data = ''
T = 20 #Rows
P = 20 #Cols

#Optional
R = int(0.07 * P) #Enforces sparcity 
M = 5 #Dimensionality of the learned dictionary
E = 0.01 #Epsilon
norm = True

print(f'R:\t{R}')
#Spark Options, look for similar Dask elements
partitions = 4*4
execmem = '8g'

#Outputs
file_D = 'D.txt'
file_z = 'z.txt'
prefix = False

R:	1


In [3]:
client = Client()

In [4]:
max_iterations = P*10
seed = np.random.randint(max_iterations +1, high = 4294967295)
S, D, SV = make_sparse_coded_signal(T,M,P,R,seed)
S = da.from_array(S)
u_new = da.zeros(T).compute()
v = da.zeros(P).compute()

In [5]:
for m in range(M):
    #We need to broadcast the seed to create the initial random vector 
    #seed = np.random.randint(max_iterations +1, high = 4294967295)
    _SEED_ = client.scatter(seed, broadcast=True)
    np.random.seed(_SEED_.result())
    
    #Create a dense random vector
    u_old = da.random.random(T)
    num_iterations = 0
    delta = 2*E
    
    while num_iterations < max_iterations and delta > E:
        
        _U_ = client.scatter(u_old, broadcast=True) 
            
        v = da.dot(_U_.result(),S) 
        print(type(v))
        #Grab the indices of the top R values in v for the sparse vector
        indices = sorted(v.argtopk(R,axis=0))
        
        #let's make the sparse vector.
        sv = sparse.COO(indices,v[indices].compute(),shape=(P),sorted=True)
        sv = da.from_array(sv)
        

        
        # Broadcast the sparse vector.
        _V_ = client.scatter(sv,broadcast=True)

        # P1: Matrix-vector multiplication step. Computes u.
        u_new = da.dot(S,_V_.result()) 
        
        
        # Subtract off the mean and normalize.
        u_new -= u_new.mean()
        u_new /= da.linalg.norm(u_new)
        u_new = u_new.compute()
        print(type(u_new))
        # Update for the next iteration.
        delta = sla.norm(u_old - u_new) #Should u_old be _U_?
        u_old = u_new
        num_iterations += 1
        
    # Save the newly-computed u and v to the output files;
    with open(file_D, "a+") as fD:
        np.savetxt(fD, u_new, fmt = "%.6f", newline = " ")
        fD.write("\n")
    with open(file_z, "a+") as fz:
        np.savetxt(fz, sv.compute().todense(), fmt = "%.6f", newline = " ")
        fz.write("\n")

    
    # P4: Deflation step. Update the primary data matrix S. 
    _U_ = client.scatter(u_new, broadcast=True)
    _V_ = client.scatter(sv, broadcast=True)

    
    #if args['debug']: print(m)
    print(m)
    print(type(_U_.result()),type(_V_.result()))
    S -= da.outer(_U_.result(),_V_.result().compute().todense())
    S.compute()

    print(datetime.datetime.now())
    #if args['debug']: print(datetime.datetime.now())
    process = psutil.Process(os.getpid())
    print(process.memory_info().rss)

<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
0
<class 'numpy.ndarray'> <class 'dask.array.core.Array'>
2020-01-27 22:58:50.973265
204259328
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
1
<class 'numpy.ndarray'> <class 'dask.array.core.Array'>
2020-01-27 22:58:51.900638
204275712
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
2
<class 'numpy.ndarray'> <class 'dask.array.core.Array'>
2020-01-27 22:58:52.765399
204275712
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
3
<class 'numpy.ndarray'> <class 'dask.array.core.Array'>
2020-01-27 22:58:53.864704
204275712
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<cla